# Python Selenium untuk Scraping Rating Data Traveloka
hotel_link_list diambil dari data txtnya alfi

In [27]:
# Download Selenium
!pip install selenium
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!gdown --id 1bDx9hdqPsz8zxf7Pmj3HCSdIxj8OPvQT


ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/comm

In [28]:
# get listOfLink from links.txt
with open('/content/links.txt') as f:
    string = f.readline()
    listOfLink = string[1:-1].replace("'", "").split(", ")

In [29]:
import sys
import time
import re
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [30]:
def getHotelIdFromLink(link):
  result = re.search(r"[a-zA-Z\/:.-]*([0-9]*)?", link)
  return result.group(1)

In [31]:
def checkEmptyString(listOfString):
  for string in listOfString:
    if string == '':
      return False
  return True

In [32]:
def createUserItem(name, rating, review, travel_purpose, user_item): 
  user_item['name'] = name
  user_item['rating'] = rating
  user_item['review'] = review
  user_item['travel_purpose'] = travel_purpose
  if not checkEmptyString([name, rating, review, travel_purpose]):
    return False
  return user_item

In [33]:
def getBlockReview(wd, hotel_id, db):
  review_container = wd.find_element_by_xpath('//div[contains(@class, "_1xx34")]')
  try :
    load_more_button = wd.find_element_by_xpath('//div[contains(@class, "_1hiwh")]')
  except:
      return;

  x_path_name = ".//div[@class='css-901oao r-1sixt3s r-1b43r93 r-b88u0q r-rjixqe r-fdjqy7']"
  x_path_rating = ".//div[@class='css-901oao r-1i6uqv8 r-1sixt3s r-1b43r93 r-majxgm r-rjixqe r-fdjqy7']"
  x_path_review = ".//div[@class='css-901oao r-1sixt3s r-1b43r93 r-majxgm r-rjixqe r-fdjqy7']"
  x_path_travel_purpose = ".//div[@class='css-901oao r-1wb8tzg r-1sixt3s r-1enofrn r-majxgm r-135wba7 r-fdjqy7']"

  total_review = 0
  temp_idx = 0

  while total_review < 14:
    review_list = review_container.find_elements_by_xpath('//div[contains(@class, "_3Kvrw")]/div')
    for i in range(temp_idx, temp_idx+10):
      try:
        id = review_list[i].get_attribute('id')
        name = review_list[i].find_element_by_xpath(x_path_name).text
        rating = review_list[i].find_element_by_xpath(x_path_rating).text
        review = review_list[i].find_element_by_xpath(x_path_review).text
        try:
          travel_purpose = review_list[i].find_element_by_xpath(x_path_travel_purpose).text
        except:
          travel_purpose = ''
        user_item = {
            'user_id': id,
            'hotel_id': hotel_id
        }
        user_item = createUserItem(name, rating, review, travel_purpose, user_item)
        if user_item :
          db.append(user_item)
          total_review += 1
          review_list[i+1]
      except:
        total_review = 15

    wd.execute_script("arguments[0].click();", load_more_button)
    time.sleep(8)
    temp_idx = i + 1

In [34]:
def getReviewHotel(wd, current_url, db):
  wd.get(current_url)
  hotel_id = getHotelIdFromLink(current_url)

  # pilih rendah ke tinggi
  rendah_ke_tinggi_button = wd.find_element_by_xpath("//div[@class='_2oYpR']/ul/li[2]")
  wd.execute_script("arguments[0].click();", rendah_ke_tinggi_button)
  time.sleep(8)

  getBlockReview(wd, hotel_id, db)

  tinggi_ke_rendah_button = wd.find_element_by_xpath("//div[@class='_2oYpR']/ul/li[3]")
  wd.execute_script("arguments[0].click();", tinggi_ke_rendah_button)
  time.sleep(8)

  getBlockReview(wd, hotel_id, db)

In [35]:
import csv
def export_csv(filename, column, data):
    """Export list of dictionary to CSV"""
    try:
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=column)
            writer.writeheader()
            writer.writerows(data)
    except IOError:
        print("I/O error")

In [37]:
db = []
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
for link in listOfLink:
  dbTemp = []
  getReviewHotel(wd, link, dbTemp)
  db += dbTemp
  print(link, dbTemp)

print()
print("semua", db)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


KeyboardInterrupt: ignored

In [ ]:
column = ['user_id', 'hotel_id', 'name', 'rating', 'review', 'travel_purpose']
export_csv('/content/review.csv', column, db)